In [1]:
from addons.stock_manage.service import DataLoader
d = DataLoader('000001.SZ', '2000-01-01', '2023-01-31')
res = d.get_stack_daily()
d.add_ema(res)
d.add_target(res)
print(res.tail())


       ts_code  trade_date   open   high    low  close  pre_close  pct_chg  \
175  000001.SZ  2023-01-17  15.13  15.18  14.77  14.97      15.08  -0.7294   
176  000001.SZ  2023-01-18  14.95  15.18  14.91  15.11      14.97   0.9352   
177  000001.SZ  2023-01-19  15.13  15.25  14.87  15.09      15.11  -0.1324   
178  000001.SZ  2023-01-20  15.16  15.24  15.00  15.13      15.09   0.2651   
179  000001.SZ  2023-01-30  15.60  15.74  14.89  15.15      15.13   0.1322   

     change         vol  ...     ema_30      ema42  buy_signal  sell_signal  \
175   -0.11   935834.54  ...  13.763968  14.509597           1            0   
176    0.14   718434.03  ...  13.850808  14.666051           1            0   
177   -0.02   641875.20  ...  13.930756  14.812596           1            0   
178    0.04   608590.08  ...  14.008127  14.951877           1            0   
179    0.02  1374317.50  ...  14.081796  15.081636           1            0   

     Open-Close  High-Low  Val_Norm  Amount_Norm  target

In [2]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
def classification_tc(df: pd.DataFrame):
    X = df[['Open-Close', 'High-Low', 'Val_Norm', 'Amount_Norm']]
    y = df["target_cls"]
    return (X, y)
X, y = classification_tc(res)
#将数据集拆分为训练集与验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
#创建一个KNN实例，n_neighbors取95
knn_clf = KNeighborsClassifier(n_neighbors=95)
#使用KNN拟合训练集
knn_clf.fit(X_train, y_train)
#输出模型在训练集中的准确率
print(knn_clf.score(X_train, y_train))
#输出模型在验证集中的准确率
print(knn_clf.score(X_test, y_test))

res['Predict_Signal'] = knn_clf.predict(X)
#在数据集中添加一个字段，用当日收盘价除以前一日收盘价，并取其自然对数
res['Return'] = np.log(res['close']/res['pre_close'])
#查看一下
print(res.tail())

0.5555555555555556
0.5
       ts_code  trade_date   open   high    low  close  pre_close  pct_chg  \
175  000001.SZ  2023-01-17  15.13  15.18  14.77  14.97      15.08  -0.7294   
176  000001.SZ  2023-01-18  14.95  15.18  14.91  15.11      14.97   0.9352   
177  000001.SZ  2023-01-19  15.13  15.25  14.87  15.09      15.11  -0.1324   
178  000001.SZ  2023-01-20  15.16  15.24  15.00  15.13      15.09   0.2651   
179  000001.SZ  2023-01-30  15.60  15.74  14.89  15.15      15.13   0.1322   

     change         vol  ...  buy_signal  sell_signal  Open-Close  High-Low  \
175   -0.11   935834.54  ...           1            0        0.16      0.41   
176    0.14   718434.03  ...           1            0       -0.16      0.27   
177   -0.02   641875.20  ...           1            0        0.04      0.38   
178    0.04   608590.08  ...           1            0        0.03      0.24   
179    0.02  1374317.50  ...           1            0        0.45      0.85   

     Val_Norm  Amount_Norm  targe

In [4]:
import torch

In [5]:
print(torch.__version__)

print(torch.version.cuda)
print(torch.backends.cudnn.version())

2.0.0+cu117
11.7
8500
